### This is the main code that performs model training (60 models) with 3 resampled distributions, 4 model architectures, and 5 datasets with various sources and number of samples

In [1]:
import sys
import gc
import os, shutil
import tempfile
from os import listdir
import clr_callback

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import seaborn as sns
from PIL import Image

from keras import backend as K

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras import mixed_precision, regularizers
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.layers import Input, Add, Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity, he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, CSVLogger
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import InceptionV3, Xception, MobileNetV3Large,EfficientNetB0,EfficientNetV2B0
from resnet import resnet18

from sklearn.metrics import classification_report,confusion_matrix, matthews_corrcoef
from sklearn.utils.class_weight import compute_class_weight

In [2]:
plt.set_cmap('gray')
pd.set_option('precision', 3)

<Figure size 432x288 with 0 Axes>

In [3]:
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

### Initial setting of several hyperparameters, as well as the use of *mixed precision* from NVIDIA CUDA/CU-DNN

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.05)

clr = clr_callback.CyclicLR(base_lr=1e-4, max_lr=5e-3,
               step_size=2000, mode='triangular2')
    
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.005, patience = 6,  restore_best_weights=True)
lrs = LearningRateScheduler(scheduler)

### Specify the dataset folder in training process (Model HQ/HP-S/HP-L/RS-S/RS-L) = (Model A-B-C-D-E)

In [ ]:
train_path = 'train_path'
val_path = 'val_path'
test_path = 'test_path'

### Pipeline Preprocessing

In [ ]:
def create_datagen(train_path, val_path, test_path, target_size = (256,256), batch_size = 16, efficient = False):
    if efficient:
        train_datagen = image.ImageDataGenerator(
            rescale = 1.,
        )
    else:
        train_datagen = image.ImageDataGenerator(
            rescale = 1./255,
        )

    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = True
    )

    validation_generator = train_datagen.flow_from_directory(
        val_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = False
    )
    
    test_generator = train_datagen.flow_from_directory(
        test_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = False
    )
    return train_generator, validation_generator, test_generator

### Utility Functions for Confusion Matrix Plotting and Simple Visualization

In [ ]:
plt.rcParams["figure.figsize"] = (9,12)

def plot_model_history(model, path):
    fig, (ax1, ax2) = plt.subplots(2)
    ax1.plot(model.history['accuracy'])
    ax1.plot(model.history['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Val'], loc='upper left')
    
    ax2.plot(model.history['loss'], 'b')
    ax2.plot(model.history['val_loss'], 'r')
    ax2.set_title('Training and Validation loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train Loss', 'Val Loss'], loc='upper left')
    fig.savefig(path, bbox_inches = 'tight')

    plt.show()

In [ ]:
def plot_confusion_matrix(df_confusion, path, title='Confusion Matrix'):
    print(df_confusion)
    plt.figure(figsize=(8, 6))
    plt.title(title)

    heatmaps = sns.heatmap(df_confusion, annot=True, cmap = "viridis",
               vmin = 0, vmax = 1)
    plt.setp(heatmaps.get_xticklabels(), rotation=30)
    plt.setp(heatmaps.get_yticklabels(), rotation=30)
    plt.tight_layout()
    plt.savefig(path)
    plt.show()

In [ ]:
def report_classifier(model,generator, testing_size, batch_size, evaluate = False, efficient_net = False, 
                      path_report = None):
    Y_pred = model.predict(generator)
    y_pred = np.argmax(Y_pred, axis=1)
    k = matthews_corrcoef(generator.classes, y_pred)
    print(f'Matthew Correlation Coefficient : {k:.2f}')
    
    df_confusion_ori = pd.crosstab(generator.classes, y_pred, 
                               rownames=['Actual'], colnames=['Predicted'], margins=True)
    df_confusion = pd.crosstab(generator.classes, y_pred, 
                               rownames=['Actual'], colnames=['Predicted'], margins=True, normalize = "index")
    df_confusion.rename(columns={0: 'Confined Masonry', 1 : 'RC Infilled', 2 : 'Timber', 3 : 'Unconfined'}, 
              index={0: 'Confined Masonry', 1 : 'RC Infilled', 2 : 'Timber', 3 : 'Unconfined'}, inplace = True)
    plot_confusion_matrix(df_confusion[:4][:], path = path_report)
    
    print('Classification Report')
    target_names = ['Confined', 'RC', 'Timber', 'Unconfined']
    print(classification_report(generator.classes, y_pred, target_names=target_names))
    
    if evaluate:
        tipologi = {0 : 'Confined', 1 : 'RC Infilled', 2 : 'Timber', 3 : 'Unconfined'}
        generator.reset()
        print_index = 0
        showimg = 1
        plt.figure(figsize=(16,8))
        while(print_index < len(y_pred)):
            x_batch, y_batch = next(generator)
            for k, (img, lbl) in enumerate(zip(x_batch, y_batch)):
                if(showimg == 4):
                    plt.figure(figsize=(16,8))
                    showimg = 1
                if (y_pred[print_index] != np.argmax(lbl)):
                    if efficient_net:
                        plt.subplot(1, 4, showimg)#4 rows with 8 images.
                        showimg += 1
                        plt.title('Prediksi :' + str(tipologi[y_pred[print_index]]) + ', Aktual :' + str(tipologi[np.argmax(lbl)]), 
                                  fontsize = 9)
                        plt.axis('off')
                        plt.tight_layout()
                        plt.imshow(img/255.)
                    else:
                        plt.subplot(1, 4, showimg)#4 rows with 8 images.
                        showimg += 1
                        plt.title('Prediksi :' + str(y_pred[print_index]) + ', Aktual :' + str(np.argmax(lbl)), 
                                  fontsize = 9)
                        plt.axis('off')
                        plt.tight_layout()
                        plt.imshow(img)
                print_index += 1
    return

In [ ]:
train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path,
                                                                       batch_size = 16,efficient = False)

### Specification of additional *class weight* coefficient to balance the unbalanced situation of the dataset (extra effort after augmentation)

In [ ]:
class_weights = compute_class_weight(
               class_weight = 'balanced',
                classes = np.unique(validation_generator.classes), 
                y = validation_generator.classes)

In [ ]:
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)

### Utility Functions for initiating deep learning models

In [ ]:
initializer = tf.keras.initializers.HeNormal()

def add_regularization(model, regularizer = regularizers.l2(0.001)):
    if not isinstance(regularizer, regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    model_json = model.to_json()

    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    model = models.model_from_json(model_json)
    
    model.load_weights(tmp_weights_path, by_name=True)
    return model

In [ ]:
def build_models():
    model = Sequential()
    model.add(Input(shape = (256,256,3)))
    model.add(Conv2D(8, 3, activation = 'relu', padding = 'valid'))
    model.add(Conv2D(8, 3, activation = 'relu', padding = 'valid'))
    model.add(Conv2D(8, 3, activation = 'relu', padding = 'valid'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Conv2D(16, 3, activation = 'relu', padding = 'valid'))
    model.add(Conv2D(16, 3, activation = 'relu', padding = 'valid'))
    model.add(Conv2D(16, 3, activation = 'relu', padding = 'valid'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Conv2D(32, 3, activation = 'relu', padding = 'valid'))
    model.add(Conv2D(32, 3, activation = 'relu', padding = 'valid'))
    model.add(Conv2D(32, 3, activation = 'relu', padding = 'valid'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Conv2D(64, 3, activation = 'relu', padding = 'valid'))
    model.add(Conv2D(64, 3, activation = 'relu', padding = 'valid'))
    model.add(Conv2D(64, 3, activation = 'relu', padding = 'valid'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(128, kernel_regularizer=regularizers.l1(0.01),
                    activity_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(128, kernel_regularizer=regularizers.l1(0.01),
                    activity_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate=1e-4, 
                                                                      beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False), 
                  metrics = ['accuracy'])
    return model

In [ ]:
model = build_models()
add_regularization(model)
model.summary(show_trainable = True)

In [ ]:
validation_generator

### Data generator experimentation

In [ ]:
x_batch, y_batch = next(train_generator)

plt.figure(figsize=(16, 32))
for k, (img, lbl) in enumerate(zip(x_batch, y_batch)):
    plt.subplot(8, 4, k+1)#4 rows with 8 images.
    plt.title(str(lbl))
    plt.imshow(img)

## Model Experimentation Function with various parameters

In [ ]:
'''
This experimental function is actively used in my study. Parameter:

model = The type of model whose performance is being reviewed through the training and validation process
train_gen = Object generator to generate photos for the training process
validation_gen = Generator object to generate photos for validation process (never seen by model)
epoch_sch = Setting the number of epochs for each cycle of the fine-tuning process
fine_tune_sch = Freeze/unfreeze layer settings for each cycle of the fine-tuning process
lr_sch = Learning rate schedule, the change in the learning rate value in each cycle of the fine-tuning process. 
        When the fine-tuning approaches the earlier layer of CNN, the learning rate used is gradually smaller
validation_sample = Number of samples in the validation set
model_name = The name of the model to be saved to local disk
label_smooth = Smoothing label parameter to provide soft label
es = Early stopping object to stop training when overfitting starts
lrs = object learning rate schedule, reducing the learning rate value at epoch > 10 negatively exponentially for
      guarantee stable convergence
spe = Steps per epoch, for certain cases where the dataset is very large, speeding up the training process with consequences
      performance
save_epoch = Cycle fine-tuning where the model starts to be saved gradually to evaluate its performance in the learning stage
             certain
'''

def experiment(model, train_gen, validation_gen, 
               epoch_sch, fine_tune_sch, lr_sch, 
               validation_sample, class_weight, model_name,
               label_smooth = 0.2, es = es, lrs = lrs, spe = None, save_epoch = 0):
    for i in range(len(epoch_sch)):
        fine_tune = fine_tune_sch[i]
        epochs = epoch_sch[i]
        learning_rates = lr_sch[i]
    
        for layer in model.layers[:fine_tune]:
            layer.trainable = False
        for layer in model.layers[fine_tune:]:
            layer.trainable = True
        model.compile(loss = CategoricalCrossentropy(from_logits=False, label_smoothing=label_smooth, axis=-1), 
                                optimizer = Adam(learning_rate=learning_rates, beta_1=0.9, beta_2=0.999, 
                                            epsilon=None, amsgrad=False), 
                               metrics = ['accuracy'])
        print(f'Training Model for {epochs} epoch, fine-tuned at {fine_tune}, and with learning rate of {learning_rates}')
        model_title = model_name + str(i+1)
        csv_logger = CSVLogger('Misc/temp Graph/'+model_title+' Report.csv', append=True)
        hist = model.fit(
            train_gen,
            epochs = epochs,
            validation_data=validation_gen,
            class_weight = class_weight,
            callbacks=[es, lrs, csv_logger],
            steps_per_epoch = spe,
            verbose = 2
        )
        plot_model_history(hist, path = 'Misc/temp Graph/'+str(model_title)+' trainlog.jpg')
        if(i >= (len(epoch_sch)-5)):
            report_classifier(model,validation_generator,validation_sample,32,
                              path_report = 'Misc/temp Graph/'+model_title+' Report.jpg')
        tf.keras.backend.clear_session()
        if(i >= save_epoch):
            model.save(r'Deep Learning Models/Typology Classifier/' + model_title + '.h5')
            gc.collect()

### Training + Testing with pretrained InceptionV3 Architecture

In [ ]:
def make_Inception():
    base = InceptionV3(
        include_top = False,
        weights = "imagenet",
        input_shape = (256, 256, 3),
        pooling = "max",
        classes = 4,
        classifier_activation="softmax",
    )
    out = Dropout(0.4)(base.output)
    out = Dense(32, activation='relu', kernel_initializer=initializer)(out)
    out = BatchNormalization()(out)
    out = Dropout(0.4)(out)
    out = Dense(4, activation='softmax', kernel_initializer="glorot_uniform")(out)

    model = Model(inputs = base.input,outputs=out)
    add_regularization(model)

    return model

In [ ]:
train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path, 
                                                                       batch_size = 16, efficient = False)

In [ ]:
model = make_Inception()

for i,layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
epoch_schedule = [10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 25]
fine_tune_schedule = [311, 299, 268, 228, 196, 164, 132, 100, 86, 63, 40,  4]
lr_schedule = [1e-3, 1.33e-4, 1.33e-4, 1.33e-4, 1.33e-4, 1e-4, 7e-5, 6e-5, 4e-5, 2e-5, 2e-5, 1e-5]

In [ ]:
experiment(model, train_generator, validation_generator,
           epoch_schedule, fine_tune_schedule, lr_schedule, 
           validation_sample = 381, class_weight = d_class_weights, 
           model_name = 'Model RSL - InceptionV3 - D3 - ',
           label_smooth = 0.2, es = es, lrs = lrs, save_epoch = 10)

In [ ]:
report_classifier(model,test_generator,388,16,
                 path_report = 'Misc/temp Graph/'+'TEST Model RSL - InceptionV3 - D3 - '+' Report.jpg')

In [ ]:
model_inception = tf.keras.models.load_model('Deep Learning Models\Typology Classifier\Model E - InceptionV3 - 2.h5')

In [ ]:
tf.keras.backend.clear_session()

### Training + Testing with pretrained Xception Architecture

In [ ]:
base = Xception(
    include_top = False,
    weights = "imagenet",
    input_shape = (256, 256, 3),
    pooling = "max"
)
out = Dropout(0.5)(base.output)
out = Dense(32, activation='relu', kernel_initializer=initializer)(out)
out = BatchNormalization()(out)
out = Dropout(0.5)(out)
out = Dense(4, activation='softmax', kernel_initializer="glorot_uniform")(out)

model = Model(inputs = base.input,outputs=out)
add_regularization(model)

In [ ]:
train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path, 
                                                                       batch_size = 16, efficient = False)

In [ ]:
epoch_schedule = [10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 25]
fine_tune_schedule = [133, 126, 115, 105, 95, 85, 75, 65, 55, 45, 35, 25, 15, 1]
lr_schedule = [2e-3, 2.66e-4,2.66e-4, 2.66e-4, 2.66e-4, 2.66e-4, 1e-4,1e-4, 9e-5, 8e-5, 5e-5, 3e-5, 2e-5, 1e-5]

In [ ]:
for i,layer in enumerate(model.layers):
    print(i, layer.name)

In [ ]:
experiment(model, train_generator, validation_generator,
           epoch_schedule, fine_tune_schedule, lr_schedule, 
           validation_sample = 381, class_weight = d_class_weights, 
           model_name = 'Model RSL - Xception - D3 - ',
           label_smooth = 0.2, es = es, lrs = lrs, save_epoch = 12)

In [ ]:
report_classifier(model,test_generator,388,16,
                 path_report = 'Misc/temp Graph/'+'TEST Model RSL - Xception - D3 - '+' Report.jpg')

In [ ]:
tf.keras.backend.clear_session()

### Training + Testing with pretrained MobileNet V3L Architecture

In [ ]:
def make_mobileNet():
    base = MobileNetV3Large(
        input_shape= (256, 256, 3),
        alpha=1.0,
        include_top = False,
        weights= "imagenet",
        classes = 4,
        pooling = "max",
        dropout_rate = 0.5,
    )
    out = Dropout(0.5)(base.output)
    out = Dense(32, activation='relu', kernel_initializer=initializer)(out)
    out = BatchNormalization()(out)
    out = Dropout(0.5)(out)
    out = Dense(4, activation='softmax', kernel_initializer="glorot_uniform")(out)

    model = Model(inputs = base.input,outputs=out)
    add_regularization(model)

    return model

In [ ]:
model = make_mobileNet()

In [ ]:
train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path,
                                                                       batch_size = 32, efficient = True)

In [ ]:
epoch_schedule = [10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 25]
fine_tune_schedule = [262, 256, 233, 210, 187, 164, 142, 127, 112, 97, 82, 65, 48, 31,22, 1]
lr_schedule = [1e-3, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 8e-5, 8e-5, 7e-5, 6e-5, 4e-5, 2e-5, 2e-5, 2e-5, 1e-5]

In [ ]:
for i,layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
experiment(model, train_generator, validation_generator,
           epoch_schedule, fine_tune_schedule, lr_schedule, 
           validation_sample = 381, class_weight = d_class_weights, 
           model_name = 'Model RSL - Mobile - D1 - ',
           label_smooth = 0.2, es = es, lrs = lrs, save_epoch = 12)

In [ ]:
report_classifier(model,test_generator,388,32,
                 path_report = 'Misc/temp Graph/'+'TEST Model RSL - Mobile - D1 - '+' Report.jpg')

In [ ]:
tf.keras.backend.clear_session()

### Training + Testing with pretrained EfficientNet B0 Architecture

In [ ]:
def make_EfficientB0():
    base = EfficientNetB0(
        include_top = False,
        weights = "imagenet",
        input_shape = (256, 256, 3),
        pooling = "max",
        classes = 4
    )
    out = Dropout(0.5)(base.output)
    out = Dense(32, activation='relu', kernel_initializer=initializer)(out)
    out = BatchNormalization()(out)
    out = Dropout(0.5)(out)
    out = Dense(4, activation='softmax', kernel_initializer="glorot_uniform")(out)

    model = Model(inputs = base.input,outputs=out)
    add_regularization(model)

    return model

In [ ]:
model = make_EfficientB0()

In [ ]:
train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path, 
                                                                       batch_size = 16, efficient = True)

In [ ]:
for i,layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)
    
model.summary(show_trainable = True)

In [ ]:
epoch_schedule = [10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 18]
fine_tune_schedule = [238, 234, 162,147, 132, 119, 104, 89, 75, 60, 46, 31, 17,3]
lr_schedule = [1e-2, 1e-3, 1e-3, 1e-3, 1e-3, 6e-4, 6e-4, 3e-4, 3e-4, 3e-4, 8e-5, 8e-5, 8e-5, 4e-5]

In [ ]:
experiment(model, train_generator, validation_generator,
           epoch_schedule, fine_tune_schedule, lr_schedule, 
           validation_sample = 101, class_weight = d_class_weights, 
           model_name = 'Model RSS - Efficient - D3 - ',
           label_smooth = 0.2, es = es, lrs = lrs, save_epoch = 12)

In [ ]:
report_classifier(model,test_generator,106,16, 
                 path_report = 'Misc/temp Graph/'+'TEST Model RSS - Efficient - D3 - '+' Report.jpg')

In [ ]:
tf.keras.backend.clear_session()